<a href="https://colab.research.google.com/github/ArielleAsher/CycleGANSkin/blob/main/CycleGAN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the keras-contrib repository
!git clone https://github.com/keras-team/keras-contrib.git

# Navigate to the cloned directory
%cd keras-contrib

# Install the library
!pip install .

# Go back to the root directory
%cd ..

Cloning into 'keras-contrib'...
remote: Enumerating objects: 3634, done.
remote: Total 3634 (delta 0), reused 0 (delta 0), pack-reused 3634
Receiving objects: 100% (3634/3634), 861.24 KiB | 2.22 MiB/s, done.
Resolving deltas: 100% (2330/2330), done.
/content/keras-contrib
Processing /content/keras-contrib
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=9facaefc5df23ca27560f3de3539e6f8c6ea931eca9696c24b2aef281ea81941
  Stored in directory: /root/.cache/pip/wheels/cc/d2/a0/84f3e4b6f9e8b528b2342c7b7952238c24b919a53a63c60144
Successfully built keras-contrib
/content


In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
!pip install tensorflow_addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import pandas as pd


drive_base_path = '/content/MyDrive'
# /content/drive/MyDrive/all_images

source_images_path = os.path.join(drive_base_path, 'MyDrive/all images')  # Source folder with images
destination_images_path = os.path.join(drive_base_path, 'MyDrive/all_images')  # Destination folder with new directories
metadata_path = os.path.join(drive_base_path, 'MyDrive/HAM10000_metadata')

# Create folders for each cancer type in the destination folder
cancer_types = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
for cancer_type in cancer_types:
    os.makedirs(os.path.join(destination_images_path, cancer_type), exist_ok=True)
    print(f"Created folder: {cancer_type}")


# Read metadata
metadata = pd.read_csv(metadata_path)

# Debug: Print the first few rows of the metadata to ensure it's read correctly
print("Metadata preview:")
print(metadata.head())

# Move images to the corresponding cancer type folders in the destination folder
for i, row in metadata.iterrows():
    image_name = row['image_id'] + '.jpg'
    cancer_type = row['dx']
    source = os.path.join(source_images_path, image_name)
    destination = os.path.join(destination_images_path, cancer_type, image_name)

    # Check if the source image exists before moving
    if os.path.isfile(source):
        shutil.copy(source, destination)
        print(f"Moved {image_name} to {destination}")
    else:
        print(f"Source image not found: {source}")

print("Images moved successfully!")

# Check the number of images in each folder in the destination folder
for cancer_type in cancer_types:
    folder_path = os.path.join(destination_images_path, cancer_type)
    num_images = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
    print(f"Cancer type '{cancer_type}' has {num_images} images.")


Streaming output truncated to the last 5000 lines.
Moved ISIC_0031828.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0031828.jpg
Moved ISIC_0025395.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0025395.jpg
Moved ISIC_0030304.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0030304.jpg
Moved ISIC_0030982.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0030982.jpg
Moved ISIC_0025445.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0025445.jpg
Moved ISIC_0024509.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0024509.jpg
Moved ISIC_0031419.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0031419.jpg
Moved ISIC_0031949.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0031949.jpg
Moved ISIC_0028996.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0028996.jpg
Moved ISIC_0029586.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0029586.jpg
Moved ISIC_0027059.jpg to /content/MyDrive/MyDrive/all_images/nv/ISIC_0027059.jpg
Moved ISIC_0028405.jpg to /content/MyDrive/MyDr

In [ ]:
# import datetime
# import glob
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, UpSampling2D, Concatenate, Dropout, Flatten, Dense, MaxPooling2D
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.preprocessing.image import img_to_array, load_img
# from tensorflow.keras.optimizers import Adam
# from sklearn.model_selection import train_test_split
# import tensorflow as tf

import datetime
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, UpSampling2D, Concatenate, Dropout, Flatten, Dense, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Configure TensorFlow to use GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

class DataLoader:
    def __init__(self, dataset_path, img_res=(128, 128)):
        self.dataset_path = dataset_path
        self.img_res = img_res

    def load_data(self, cancer_type, batch_size=1, is_testing=False):
        path = os.path.join(self.dataset_path, cancer_type)
        images = glob.glob(path + '/*.jpg')
        if len(images) == 0:
            images = glob.glob(path + '/*.png')

        batch_images = np.random.choice(images, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = self.augment(img)
            imgs.append(img)
        imgs = np.array(imgs) / 127.5 - 1.
        return imgs

    def load_batch(self, cancer_type_A, cancer_type_B, batch_size=1, is_testing=False):
        path_A = os.path.join(self.dataset_path, cancer_type_A)
        path_B = os.path.join(self.dataset_path, cancer_type_B)
        images_A = glob.glob(path_A + '/*.jpg')
        if len(images_A) == 0:
            images_A = glob.glob(path_A + '/*.png')
        images_B = glob.glob(path_B + '/*.jpg')
        if len(images_B) == 0:
            images_B = glob.glob(path_B + '/*.png')

        self.n_batches = int(min(len(images_A), len(images_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        for i in range(self.n_batches - 1):
            batch_A = images_A[i*batch_size:(i+1)*batch_size]
            batch_B = images_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_path in batch_A:
                img = self.imread(img_path)
                if not is_testing:
                    img = self.augment(img)
                imgs_A.append(img)
            for img_path in batch_B:
                img = self.imread(img_path)
                if not is_testing:
                    img = self.augment(img)
                imgs_B.append(img)
            imgs_A = np.array(imgs_A) / 127.5 - 1.
            imgs_B = np.array(imgs_B) / 127.5 - 1.
            yield imgs_A, imgs_B

    def imread(self, path):
        img = load_img(path, target_size=self.img_res)
        img = img_to_array(img)
        return img

    def augment(self, img):
        # Add any image augmentation here if needed
        return img

class CycleGAN():
    def __init__(self, dataset_path, img_res=(128, 128)):
        self.img_rows = img_res[0]
        self.img_cols = img_res[1]
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        self.dataset_path = dataset_path
        self.data_loader = DataLoader(dataset_path=self.dataset_path,
                                      img_res=(self.img_rows, self.img_cols))

        self.disc_patch = (self.img_rows // 2**4, self.img_cols // 2**4, 1)

        self.gf = 32
        self.df = 64

        self.lambda_cycle = 10.0
        self.lambda_id = 0.1 * self.lambda_cycle

        # optimizer = Adam(0.0002, 0.5)
        optimizer = tf.keras.optimizers.legacy.Adam(0.0002, 0.5)

        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        self.d_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        self.d_A.trainable = False
        self.d_B.trainable = False

        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[valid_A, valid_B, reconstr_A, reconstr_B, img_A_id, img_B_id])
        self.combined.compile(loss=['mse', 'mse', 'mae', 'mae', 'mae', 'mae'],
                              loss_weights=[1, 1, self.lambda_cycle, self.lambda_cycle, self.lambda_id, self.lambda_id],
                              optimizer=optimizer)

    def build_generator(self):
        def conv2d(layer_input, filters, f_size=4):
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            d = BatchNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        d0 = Input(shape=self.img_shape)
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf * 2)
        d3 = conv2d(d2, self.gf * 4)
        d4 = conv2d(d3, self.gf * 8)

        u1 = deconv2d(d4, d3, self.gf * 4)
        u2 = deconv2d(u1, d2, self.gf * 2)
        u3 = deconv2d(u2, d1, self.gf)
        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

        return Model(d0, output_img)

    def build_discriminator(self):
        def d_layer(layer_input, filters, f_size=4, normalization=True):
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = BatchNormalization()(d)
            return d

        img = Input(shape=self.img_shape)
        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df * 2)
        d3 = d_layer(d2, self.df * 4)
        d4 = d_layer(d3, self.df * 8)
        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=1, sample_interval=50, maxb=None):
        base_cancer_type = 'nv'
        other_cancer_types = ['bkl']  # Only training for nv and bkl now
        start_time = datetime.datetime.now()

        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for cancer_type in other_cancer_types:
            print(f"Training on {base_cancer_type} and {cancer_type}...")
            for epoch in range(epochs):
                for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(base_cancer_type, cancer_type, batch_size)):
                    if maxb and batch_i >= maxb:
                        break
                    fake_B = self.g_AB.predict(imgs_A)
                    fake_A = self.g_BA.predict(imgs_B)

                    dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                    dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                    dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                    dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                    dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                    dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                    d_loss = 0.5 * np.add(dA_loss, dB_loss)

                    g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                          [valid, valid, imgs_A, imgs_B, imgs_A, imgs_B])

                    elapsed_time = datetime.datetime.now() - start_time

                    print(f"[Epoch {epoch}/{epochs}] [Batch {batch_i}] [D loss: {d_loss[0]}, acc: {100*d_loss[1]}] [G loss: {g_loss}] time: {elapsed_time}")

                    if batch_i % sample_interval == 0:
                        self.sample_images(epoch, batch_i, base_cancer_type, cancer_type)

    def sample_images(self, epoch, batch_i, base_cancer_type, cancer_type):
        os.makedirs(f"images/{base_cancer_type}_{cancer_type}", exist_ok=True)
        imgs_A = self.data_loader.load_data(base_cancer_type, batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(cancer_type, batch_size=1, is_testing=True)

        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)

        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original A', 'Translated B', 'Reconstructed A', 'Original B', 'Translated A', 'Reconstructed B']
        fig, axs = plt.subplots(2, 3)
        cnt = 0
        for i in range(2):
            for j in range(3):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[cnt])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig(f"images/{base_cancer_type}_{cancer_type}/{epoch}_{batch_i}.png")
        plt.close()

def load_real_dataset(dataset_path, img_res=(128, 128), cancer_types=['nv', 'bkl']):
    data_loader = DataLoader(dataset_path, img_res)
    imgs, labels = [], []
    for i, cancer_type in enumerate(cancer_types):
        img_paths = glob.glob(os.path.join(dataset_path, cancer_type, '*.jpg'))
        if len(img_paths) == 0:
            img_paths = glob.glob(os.path.join(dataset_path, cancer_type, '*.png'))
        for img_path in img_paths:
            img = data_loader.imread(img_path)
            imgs.append(img / 127.5 - 1.)
            labels.append(i)
    return np.array(imgs), np.array(labels)

def build_classifier(img_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=img_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def main():
    dataset_path = '/content/MyDrive/MyDrive/all_images'  # Change this to your dataset path
    gan = CycleGAN(dataset_path)
    gan.train(epochs=20, batch_size=1, sample_interval=50, maxb=800)

    # Load real dataset
    imgs, labels = load_real_dataset(dataset_path)
    X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42)

    # Train classifier on the real dataset
    classifier = build_classifier(img_shape=(128, 128, 3), num_classes=2)
    classifier.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

    # Evaluate classifier
    test_loss, test_acc = classifier.evaluate(X_test, y_test)
    print(f'Test accuracy: {test_acc}')

if __name__ == '__main__':
    main()

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 55ms/step
[Epoch 5/20] [Batch 137] [D loss: 0.12383322790265083, acc: 86.328125] [G loss: [6.070280075073242, 0.28119128942489624, 0.33042076230049133, 0.11030379682779312, 0.38301190733909607, 0.15238167345523834, 0.37312930822372437]] time: 0:32:05.693394
1/1 [==============================] - 0s 26ms/step
[Epoch 5/20] [Batch 138] [D loss: 0.07000980339944363, acc: 93.359375] [G loss: [3.592787265777588, 0.12839096784591675, 0.15259912610054016, 0.15403658151626587, 0.1434897482395172, 0.18350984156131744, 0.15302416682243347]] time: 0:32:06.027540
1/1 [==============================] - 0s 25ms/step
[Epoch 5/20] [Batch 139] [D loss: 0.10940699046477675, acc: 84.765625] [G loss: [4.891402721405029, 0.17240454256534576, 0.05689505115151405, 0.3154839873313904, 0.1059892550110817, 0.3172794580459595, 0.1300911158323288]] time: 0:32:06.382347
1/1 [==============================] - 0s 25ms/step
[E